In [ ]:
########## Generate GPT responses ##########

# Install and import necessary libraries
!pip install openai
import os
import zipfile
import pandas as pd
from pathlib import Path
import shutil
import base64
from IPython.display import display, HTML
from openai import OpenAI
from google.colab import files
from PIL import Image

# Set up the OpenAI API client with your API key
client = OpenAI(api_key='**********************************')

def read_text_file(file_path):
    with open(file_path, 'r', encoding='utf-8') as file:
        return file.read()

def read_csv_file(file_path):
    encodings = ['utf-8', 'latin1', 'ISO-8859-1']
    for encoding in encodings:
        try:
            return pd.read_csv(file_path, encoding=encoding)
        except UnicodeDecodeError:
            continue
    raise UnicodeDecodeError("Could not decode file using available encodings.")

#  Generate GPT responses based on provided question, options, and text content
def generate_response(question, options, text_content):
    options_str = '\n'.join([f"{i+1}. {option}" for i, option in enumerate(options)])

    # Query 1: Ask the GPT model to select the correct answer with an explanation
    messages1 = [
        {"role": "system", "content": "You are a specialist clinician. The data provided is about one patient, and it might involve a rare disease, so consider many possibilities. As a clinician, you must make data-based decisions, highly recommended to be based on journals and medical textbooks. Examination findings are very important!"},
        {"role": "user", "content": [
            {"type": "text", "text": f"Question: {question}\n\nOptions:\n{options_str}\n\nText Content: {text_content}"}
        ]}
    ]

    response1 = client.chat.completions.create(
        model="gpt-4-turbo",
        seed=1234,
        temperature = 0.1,
        messages=messages1,
        max_tokens=150
    )

    # Query 2:  Ask the GPT model to format the response as a list of probabilities for each option
    messages2 = [
        {"role": "system", "content": "You are a data organizer. Read the answer statement carefully."},
        {"role": "user", "content": [
            {"type": "text", "text": f"Question: {question}\n\nOptions:\n{options_str}\n\nAnswer: {response1}\n\n According to the answer statement, please write 100 for only one option that is expected to be the correct answer and write 0 for the others. For example: 0, 100, 0, 0. Ensure that the total has to be 100%. NEVER include additional texts."}
        ]}
    ]

    response2 = client.chat.completions.create(
        model="gpt-4-turbo",
        seed=1234,
        temperature = 0.1,
        messages=messages2,
        max_tokens=150
    )

    return response2.choices[0].message.content.strip()

# Parse response and check percentage values
def parse_response(response_text, num_options):
    try:
        percentages = [float(p.strip()) for p in response_text.split(',')]
    except ValueError:
        return []

    total_percentage = sum(percentages)
    return [p / total_percentage * 100 for p in percentages] if total_percentage > 0 else []

# Process individual folders containing text and csv
def process_folder(folder_path):
    text_file = None
    csv_file = None

    # Iterate over all files in the folder
    for file in folder_path.iterdir():
        if file.suffix == '.txt':
            text_file = file
        elif file.suffix == '.csv':
            csv_file = file

    if not csv_file or not text_file:
        return

    text_content = read_text_file(text_file)
    df = read_csv_file(csv_file)
    question = df.columns[0]
    options = df.iloc[:, 0].dropna().tolist()
    response_text = generate_response(question, options, text_content)
    percentages = parse_response(response_text, len(options))

    # Update or add 'Percentages' column with response values
    if 'Percentages' not in df.columns:
        df['Percentages'] = pd.Series(dtype='object')

    if percentages:
        for i, percentage in enumerate(percentages):
            if i < len(df):
                df.at[i, 'Percentages'] = f"{percentage:.2f}%"
            else:
                new_row = [None] * len(df.columns)
                df.loc[len(df)] = new_row
                df.at[len(df)-1, 'Percentages'] = f"{percentage:.2f}%"
    else:
        print("No valid percentages returned by the API.")
        print(folder_path)

    # Save updated DataFrame to output.csv
    output_path = folder_path / "output.csv"
    df.to_csv(output_path, index=False)

# Process ZIP file and save results in a new ZIP file
def process_zip(zip_path, output_zip_path):
    with zipfile.ZipFile(zip_path, 'r') as zip_ref:
        zip_ref.extractall("temp_extracted")

    extracted_folder = Path("temp_extracted")
    for folder in extracted_folder.iterdir():
        if folder.is_dir():
            process_folder(folder)

    with zipfile.ZipFile(output_zip_path, 'w') as zip_out:
        for folder in extracted_folder.iterdir():
            if folder.is_dir():
                output_csv = folder / "output.csv"
                if output_csv.exists():
                    zip_out.write(output_csv, output_csv.relative_to(extracted_folder))

    shutil.rmtree("temp_extracted")

# Upload zip, process, and download results
uploaded = files.upload()
for filename in uploaded.keys():
    process_zip(filename, "output.zip")
files.download("output.zip")

In [ ]:
########## Analyze answers of Human Experts and GPT ##########

# Import necessary libraries
import zipfile
import pandas as pd
from pathlib import Path
import shutil
from google.colab import files

# Calculate accuracy ratios based on model outputs
def calculate_ratios(output_folder):
    summary_data = []
    correct_count_people = 0
    correct_count_chat = 0
    total_files = 0
    nan_percentage_files = 0

    # Iterate over each "output.csv" file in the output folder
    for file in output_folder.glob('**/output.csv'):
        df = pd.read_csv(file)

        # Check required columns and process percentage columns
        if 'Unnamed: 2' in df.columns and 'Percentages' in df.columns and 'Unnamed: 1' in df.columns:
            df['Unnamed: 1'] = df['Unnamed: 1'].astype(str).str.rstrip('%').astype('float')
            df['Percentages'] = df['Percentages'].astype(str).str.rstrip('%').astype('float')

            # Track files where 'Percentages' column is all NaN
            if df['Percentages'].isna().all():
                nan_percentage_files += 1
                print(f"File with all NaN 'Percentages' column: {file}")

            # Determine the answer selected by human experts based on maximum percentage
            df['Answer_people'] = 0
            if not df['Unnamed: 1'].isna().all():
                max_value_people = df['Unnamed: 1'].max()
                df.loc[df['Unnamed: 1'] == max_value_people, 'Answer_people'] = 1

            # Determine the answer selected by GPT based on maximum percentage
            df['Answer_chat'] = 0
            if not df['Percentages'].isna().all():
                max_value_chat = df['Percentages'].max()
                df.loc[df['Percentages'] == max_value_chat, 'Answer_chat'] = 1

            # Check if the human experts' selected answer matches the correct answer
            correct_people = 1 if df['Unnamed: 2'].equals(df['Answer_people']) else 0
            if correct_people:
                correct_count_people += 1

            # Check if the GPT's selected answer matches the correct answer
            correct_chat = 1 if df['Unnamed: 2'].equals(df['Answer_chat']) else 0
            if correct_chat:
                correct_count_chat += 1

            # Append results for the current file to summary data
            folder_name = file.parent.name
            summary_data.append([folder_name, correct_people, correct_chat])
            total_files += 1

    # Calculate accuracy ratios for human experts and GPT
    correct_ratio_people = correct_count_people / total_files if total_files > 0 else 0
    correct_ratio_chat = correct_count_chat / total_files if total_files > 0 else 0

    # Display and save summary results
    print(f"Total number of quizzes: {total_files}")
    print(f"Correct Ratio (People): {correct_ratio_people:.4f}")
    print(f"Correct Ratio (Chat): {correct_ratio_chat:.4f}")
    print(f"Number of files with 'Percentages' column all NaN: {nan_percentage_files}")

    summary_df = pd.DataFrame(summary_data, columns=['name', 'people', 'chat'])
    summary_df.to_csv('summary_results.csv', index=False)

# Process the uploaded ZIP file
def process_zip(zip_path):
    with zipfile.ZipFile(zip_path, 'r') as zip_ref:
        zip_ref.extractall("temp_extracted")

    extracted_folder = Path("temp_extracted")
    calculate_ratios(extracted_folder)
    shutil.rmtree("temp_extracted")

# Upload ZIP file, process, and download results
uploaded = files.upload()
for filename in uploaded.keys():
    process_zip(filename)
files.download('summary_results.csv')